In [6]:
import pandas as pd
import datetime

In [18]:
# Import Data:

# 1. Main Dataset
raw = pd.read_csv('/home/harry/Projects/Covid/Covid.csv')

# 2. Hospital Capacity Dataset (Source: https://healthdata.gov/dataset/covid-19-reported-patient-impact-and-hospital-capacity-state-timeseries/resource/3ce11e2c)
capacities = pd.read_csv('reported_hospital_utilization_timeseries.csv')

In [21]:
capacities.head()

,state,date,inpatient_beds_utilization,month
0,AK,2020-12-13,0.576000,12
1,AL,2020-12-13,0.751389,12
2,AR,2020-12-13,0.725288,12
3,AZ,2020-12-13,0.766585,12
4,CA,2020-12-13,0.764146,12


In [22]:
raw.head()

,cdc_report_dt,pos_spec_dt,onset_dt,current_status,sex,age_group,Race and ethnicity (combined),hosp_yn,icu_yn,death_yn,medcond_yn,month,Capacity
0,2020/11/10,2020/11/10,NaN,Laboratory-confirmed case,Male,10 - 19 Years,"Black, Non-Hispanic",No,Unknown,No,No,11,0.688734
1,2020/11/14,2020/11/10,2020/11/10,Laboratory-confirmed case,Male,10 - 19 Years,"Black, Non-Hispanic",No,No,No,No,11,0.688734
2,2020/11/19,2020/11/10,2020/11/09,Laboratory-confirmed case,Male,10 - 19 Years,"Black, Non-Hispanic",No,No,No,No,11,0.688734
3,2020/11/14,2020/11/10,NaN,Laboratory-confirmed case,Male,10 - 19 Years,"Black, Non-Hispanic",Missing,Missing,No,Missing,11,0.688734
4,2020/11/13,2020/11/10,2020/11/10,Laboratory-confirmed case,Male,10 - 19 Years,"Black, Non-Hispanic",No,No,No,Yes,11,0.688734


In [16]:
# Aggregate Capacity by month
capacities['month'] = capacities['date'].apply(lambda x: x[5:7])

months = pd.DataFrame(capacities.groupby('month').mean()['inpatient_beds_utilization'])
months = months.rename(columns={'inpatient_beds_utilization': 'Capacity'})
months

,Capacity
month,
01,0.424642
02,0.461779
03,0.518818
04,0.510773
05,0.600794
06,0.637689
07,0.664927
08,0.663529
09,0.661602


In [20]:
# Merge
raw['month'] = raw['cdc_report_dt'].apply(lambda x: x[5:7])
raw = pd.merge(raw, months, left_on = 'month', right_index=True, how='inner')
raw

,cdc_report_dt,pos_spec_dt,onset_dt,current_status,sex,age_group,Race and ethnicity (combined),hosp_yn,icu_yn,death_yn,medcond_yn,month,Capacity
0,2020/11/10,2020/11/10,NaN,Laboratory-confirmed case,Male,10 - 19 Years,"Black, Non-Hispanic",No,Unknown,No,No,11,0.688734
1,2020/11/14,2020/11/10,2020/11/10,Laboratory-confirmed case,Male,10 - 19 Years,"Black, Non-Hispanic",No,No,No,No,11,0.688734
2,2020/11/19,2020/11/10,2020/11/09,Laboratory-confirmed case,Male,10 - 19 Years,"Black, Non-Hispanic",No,No,No,No,11,0.688734
3,2020/11/14,2020/11/10,NaN,Laboratory-confirmed case,Male,10 - 19 Years,"Black, Non-Hispanic",Missing,Missing,No,Missing,11,0.688734
4,2020/11/13,2020/11/10,2020/11/10,Laboratory-confirmed case,Male,10 - 19 Years,"Black, Non-Hispanic",No,No,No,Yes,11,0.688734
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8308611,2020/02/29,NaN,2020/02/29,Laboratory-confirmed case,Female,30 - 39 Years,"White, Non-Hispanic",Unknown,Missing,No,Missing,02,0.461779
8308921,2020/02/17,NaN,2020/02/17,Laboratory-confirmed case,Female,30 - 39 Years,"White, Non-Hispanic",No,Missing,No,Missing,02,0.461779
8310683,2020/02/12,NaN,2020/02/12,Probable Case,Female,30 - 39 Years,"White, Non-Hispanic",Unknown,Missing,Unknown,Unknown,02,0.461779
8328411,2020/02/01,NaN,2020/02/01,Probable Case,Female,30 - 39 Years,"White, Non-Hispanic",No,Missing,No,Yes,02,0.461779


In [23]:
# Save
raw.to_csv('/home/harry/Projects/Covid/Covid.csv')